# 1

SMT -> FOTS


In [6]:
from z3 import *


Modes, (Start, On, Off) = EnumSort('Modes',['Start', 'On','Off'])
Events, (start, turnOn , turnOff , alive , dead) = EnumSort('Events',['start', 'turnOn', 'turnOff' , 'alive' , 'dead'])

N = 10

temp = RealVector('x', N+1)
time = RealVector('t', N+1)

modes = AstVector(); modes.resize(N+1)
events = AstVector(); events.resize(N+1)

for k in range(N+1):
    modes[k] = Const('loc_' + str(k), Modes)
    events[k] = Const('ev_'+str(k), Events)
    
def init(S):
    (X,T,M) = S
    return And(T == 0, M == Start)

def inv(S):
    (X,T,M) = S
    return If(M == On, X <= 22 , X >= 18)

def timed(S,S_,E):
    (X,T,M) = S ; (X_,T_,M_) = S_
    return And(
                T_ == T ,
                Implies( M == On , And( M_ == Off , E == turnOff , X_ > 21)),
                Implies( M == Off , And( M_ == On , E == turnOn , X_ < 19)),
                Implies( M == Start , And( M_ == Off , E == start , X_ == 20))
              )

def untimed(S,S_,E):
    (X,T,M) = S ; (X_,T_,M_) = S_
    return And(
                T_ == T ,
                Implies( M == On , And( M_ == Off , E == turnOff , X_ > 21)),
                Implies( M == Off , And( M_ == On , E == turnOn , X_ < 19)),
                Implies( M == Start , And( M_ == Off , E == start , X_ == 20))
              )

def trans(S,S_,E):
    return Or(untimed(S,S_,E) , timed(S,S_,E))

def S(k):
    return (temp[k],time[k],modes[k])
def E(k):
    return events[k]

s = Solver()
s.add(init(S(0)))

for k in range(N):
    s.add(inv(S(k)))
    s.add(trans(S(k),S(k+1),E(k)))

s.add(inv(S(N)))

print(s.check())

try:
    print(s.model())
except Z3Exception as ex:
    print("failed: %s" % ex)


sat
[loc_3 = Off,
 x__0 = 18,
 t__5 = 0,
 ev_3 = turnOn,
 x__3 = 22,
 t__0 = 0,
 x__2 = 0,
 x__1 = 20,
 t__4 = 0,
 x__5 = 22,
 t__7 = 0,
 x__4 = 0,
 x__8 = 0,
 ev_7 = turnOn,
 x__7 = 22,
 loc_5 = Off,
 t__6 = 0,
 loc_1 = Off,
 loc_0 = Start,
 loc_4 = On,
 t__1 = 0,
 ev_2 = turnOff,
 t__8 = 0,
 ev_4 = turnOff,
 loc_6 = On,
 x__10 = 0,
 ev_6 = turnOff,
 ev_8 = turnOff,
 loc_2 = On,
 t__3 = 0,
 loc_9 = Off,
 loc_8 = On,
 ev_1 = turnOn,
 t__9 = 0,
 loc_10 = On,
 x__9 = 22,
 t__2 = 0,
 x__6 = 0,
 t__10 = 0,
 ev_9 = turnOn,
 ev_0 = start,
 loc_7 = Off,
 ev_5 = turnOn]


# 2

